In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
from typing import List

In [2]:
# from np.random import seed
# seed(1)
# from tensorflow import set_random_seed
# set_random_seed(2)

tf.random.set_seed(2) 

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:

df = pd.read_csv('data_set.csv', index_col='Date')

In [5]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA12,MA26,MA50,...,% day (ma12&ma50) (cd 12&50),% day (ma12&ma100) (cd 12&100),% day (ma12&ma200) (cd 12&200),EIS,EE centerline,EE myvar,EE myvars,EE mymov,EE Above,EE Belowe
Date,,,,,,,,,,,,,,,,,,,,,
1993-11-11,27.799976,27.874707,27.706562,27.725245,88900,0.0,0,27.781295,27.797104,27.643933,...,0.496899,2.002303,3.719210,1,27.797104,0.077602,0.006022,0.307292,28.411687,27.182521
1993-11-12,27.781308,27.949453,27.762625,27.856039,108200,0.0,0,27.784410,27.803572,27.646631,...,0.498357,1.966286,3.692839,2,27.803572,0.145881,0.021281,0.307538,28.418648,27.188496
1993-11-15,27.912078,27.912078,27.762616,27.837347,243300,0.0,0,27.790637,27.816506,27.654544,...,0.492120,1.930584,3.675571,2,27.816506,0.095572,0.009134,0.307678,28.431863,27.201150
1993-11-16,27.893405,27.986819,27.781308,27.968136,492600,0.0,0,27.782854,27.828003,27.661340,...,0.439289,1.853965,3.609263,2,27.828003,0.158815,0.025222,0.308037,28.444077,27.211930
1993-11-17,27.986798,27.986798,27.743922,27.818653,39600,0.0,0,27.784411,27.841656,27.673718,...,0.399991,1.813010,3.578146,2,27.841656,0.145142,0.021066,0.307174,28.456003,27.227308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-16,436.010010,436.059998,430.920013,431.339996,75784700,0.0,0,433.032499,427.506789,422.004394,...,2.613268,5.471936,12.231187,2,427.506789,8.553208,73.157372,9.197160,445.901110,409.112468
2021-07-19,426.190002,431.410004,421.970001,424.970001,147987000,0.0,0,432.947500,427.684068,422.238714,...,2.536192,5.342504,12.068760,1,427.684068,3.725936,13.882599,9.176824,446.037715,409.330420
2021-07-20,425.679993,432.420013,424.829987,431.059998,99501600,0.0,0,432.681666,427.794193,422.381898,...,2.438497,5.181050,11.867305,1,427.794193,4.625820,21.398215,9.166508,446.127208,409.461178


In [6]:
# code used with https://colab.research.google.com/drive/1MW1s8N5Q2jNOVxo5juYww-r1ET7-wi32?usp=sharing
# https://www.youtube.com/watch?v=LI94ZkjE_w4

In [7]:
df.shape

(6973, 35)

In [8]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA12,MA26,MA50,...,% day (ma12&ma50) (cd 12&50),% day (ma12&ma100) (cd 12&100),% day (ma12&ma200) (cd 12&200),EIS,EE centerline,EE myvar,EE myvars,EE mymov,EE Above,EE Belowe
Date,,,,,,,,,,,,,,,,,,,,,
1993-11-11,27.799976,27.874707,27.706562,27.725245,88900,0.0,0,27.781295,27.797104,27.643933,...,0.496899,2.002303,3.719210,1,27.797104,0.077602,0.006022,0.307292,28.411687,27.182521
1993-11-12,27.781308,27.949453,27.762625,27.856039,108200,0.0,0,27.784410,27.803572,27.646631,...,0.498357,1.966286,3.692839,2,27.803572,0.145881,0.021281,0.307538,28.418648,27.188496
1993-11-15,27.912078,27.912078,27.762616,27.837347,243300,0.0,0,27.790637,27.816506,27.654544,...,0.492120,1.930584,3.675571,2,27.816506,0.095572,0.009134,0.307678,28.431863,27.201150
1993-11-16,27.893405,27.986819,27.781308,27.968136,492600,0.0,0,27.782854,27.828003,27.661340,...,0.439289,1.853965,3.609263,2,27.828003,0.158815,0.025222,0.308037,28.444077,27.211930
1993-11-17,27.986798,27.986798,27.743922,27.818653,39600,0.0,0,27.784411,27.841656,27.673718,...,0.399991,1.813010,3.578146,2,27.841656,0.145142,0.021066,0.307174,28.456003,27.227308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-16,436.010010,436.059998,430.920013,431.339996,75784700,0.0,0,433.032499,427.506789,422.004394,...,2.613268,5.471936,12.231187,2,427.506789,8.553208,73.157372,9.197160,445.901110,409.112468
2021-07-19,426.190002,431.410004,421.970001,424.970001,147987000,0.0,0,432.947500,427.684068,422.238714,...,2.536192,5.342504,12.068760,1,427.684068,3.725936,13.882599,9.176824,446.037715,409.330420
2021-07-20,425.679993,432.420013,424.829987,431.059998,99501600,0.0,0,432.681666,427.794193,422.381898,...,2.438497,5.181050,11.867305,1,427.794193,4.625820,21.398215,9.166508,446.127208,409.461178


In [9]:
# from sklearn.model_selection import train_test_split
# X_train, X_valid = train_test_split(df, train_size=0.8, test_size=0.2)

In [10]:
# X_train.sort_index

In [11]:
# X_valid.sort_index

In [12]:
# df[['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
#        'MA12', 'MA26', 'MA50', 'MA100', 'MA200', 'MA13', 'std12', 'std26',
#        'std50', 'std100', 'std200', '% day (o&c)', '% day (l&h)',
#        '% change (o&c) & (l&h)', '% day (ma12&ma26) (cd 12&26)',
#        '% day (ma26&ma50) (cd 26&50)', '% day (ma50&ma100) (cd 50&100)',
#        '% day (ma100&ma200) (cd 100&200)', '% day (ma12&ma50) (cd 12&50)',
#        '% day (ma12&ma100) (cd 12&100)', '% day (ma12&ma200) (cd 12&200)',
#        'EIS', 'EE centerline', 'EE myvar', 'EE myvars', 'EE mymov', 'EE Above',
#        'EE Belowe']].corr()

In [13]:
df = df.reset_index()

In [14]:
# Column Date assign type datetime64[ns]
from datetime import datetime
from functools import partial
to_datetime_fmt = partial(pd.to_datetime, format='%Y-%M-%d')
df['Date'] = df['Date'].apply(to_datetime_fmt)

In [15]:
# print(df.dtypes)
# df

In [16]:
train_size = int(df.shape[0] * 0.8)
train_df = df.iloc[:train_size]
val_df = df.iloc[train_size:]

In [17]:
train_df.shape, val_df.shape

((5578, 36), (1395, 36))

In [18]:
train_df["Date"].min(), train_df["Date"].max(), val_df["Date"].min(), val_df["Date"].max()

(Timestamp('1993-01-01 00:12:00'),
 Timestamp('2016-01-06 00:01:00'),
 Timestamp('2016-01-01 00:02:00'),
 Timestamp('2021-01-31 00:03:00'))

In [19]:
scaler = StandardScaler()
scaler.fit(train_df[["Close"]])

def make_dataset(
    df,
     window_size, 
     batch_size,
     use_scaler=True,
     shuffle=True
     ):
  features = df[["Close"]].iloc[:-window_size]
  if use_scaler:
    features = scaler.transform(features)
  data = np.array(features, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=df["Close"].iloc[window_size:],
      sequence_length=window_size,
      sequence_stride=1,
      shuffle=shuffle,
      batch_size=batch_size)
  return ds

In [20]:
window_size = 10
batch_size = 8
train_ds = make_dataset(df=train_df, window_size=window_size, batch_size=batch_size, use_scaler=True, shuffle=True)
val_ds = make_dataset(df=val_df, window_size=window_size, batch_size=batch_size, use_scaler=True, shuffle=True)

AttributeError: module 'tensorflow_core.keras.preprocessing' has no attribute 'timeseries_dataset_from_array'

In [ ]:
def compile_and_fit(model, train_ds, val_ds, num_epochs: int = 20):
  model.compile(
        loss=tf.losses.MeanSquaredError(),
        optimizer=tf.optimizers.Adam(),
        metrics=[tf.metrics.MeanAbsoluteError()]
      )
  history = model.fit(
      train_ds, 
      epochs=num_epochs,
      validation_data=val_ds,
      verbose=0
      )
  return history

In [ ]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

history =  compile_and_fit(lstm_model, train_ds, val_ds, num_epochs=200)

In [ ]:
lstm_model.evaluate(train_ds)

In [ ]:
lstm_model.evaluate(val_ds)

In [ ]:
plt.plot(history.history['mean_absolute_error'])

In [ ]:
plt.plot(history.history['val_mean_absolute_error'])

In [ ]:
q = lstm_model.predict(val_ds)
print(q.shape)

q = pd.DataFrame(q)
q.index = q.index + 5597
# q = q.rename({'0': 'Close Predict'}, axis=1)

q

In [ ]:
df2 = val_df.iloc[19:,]
df2['Close Predict'] = q
pd.options.display.max_rows = None # show all rows when they are not visible
df2['Close Predict'].sort_values()